#CLASIFICADOR DE IMAGENES PARA DETECTE LA NEUMONÍA


#####Con este modelo se pretende crear un clasificador de imágenes que sea capaz de detectar la neumonía. Para ello, se hará uso de la librería Fast.ai. 
#####Se ha utilizado el modelo AlexNet para crear este modelo. 
#####Se recomienda entrenarlo en Colab.

##### Dataset recuperado en: https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia/code 

##Conectar con google drive

In [ ]:
import sys
def in_colab():
    "Check if the code is running in Google Colaboratory"
    return 'google.colab' in sys.modules

In [ ]:
if in_colab():
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


##Instalación de dependencias

In [ ]:
if in_colab():
  !pip3 install --upgrade fastai > /dev/null

In [ ]:
import fastai
import torch
f"fastai: {fastai.__version__}, pytorch: {torch.__version__}"

'fastai: 2.3.1, pytorch: 1.8.1+cu101'

#IMPORTACIONES

In [ ]:
from fastai.vision.all import *
from pathlib import Path
from fastai.vision.widgets import *
from tqdm.auto import tqdm
import shutil

###GPU setup

La siguiente celda nos ayudará a asegurarnos que la GPU este disponible

In [ ]:
assert torch.cuda.is_available(), "CUDA no es accesible!!!"

#OPTIONS

Asegura que tienes guardada la carpeta tfg en el drive. Formato: drive / tfg / archive.zip

In [ ]:
archive = Path('/content/drive/MyDrive/tfg/archive.zip')

In [ ]:
assert archive.exists(), "No tienes guardado el archivo.zip donde se ha especificado"

Con tal de ahorrar espacio y que el programa funcione más rápido, se guardará la carpeta con el dataset comprimido en formato zip.

Cada vez que se ejecute el programa se descomprimirá el archivo y se guardará en la maquina local.

In [ ]:
!unzip '/content/drive/MyDrive/tfg/archive.zip' > /dev/null

Estudio del dataset, observo que las imagenes estan clasificadas por carpetas de test, train y val. Fast.ai ya hace la separación de train y val, por lo tanto, se reestructura el dataset según las especificaciones y se cambia el nombre de las imagenes.

In [ ]:
IMAGE_PATH = Path('/content/Imagenes Pulmones')
IMAGE_PATH.mkdir(parents=True, exist_ok=True)
IMAGES_FOR_MODEL = IMAGE_PATH / 'Imagenes modelo' #poner IMAGES en ve de IMAGE

In [ ]:
IM_PATH_NORMAL = IMAGES_FOR_MODEL / 'NORMAL'
IM_PATH_NORMAL.mkdir(parents=True, exist_ok=True)
IM_PATH_NEUMONIA = IMAGES_FOR_MODEL / 'NEUMONIA'
IM_PATH_NEUMONIA.mkdir(parents=True, exist_ok=True)

La carpeta chest_test se utilizará para comprobar el modelo creado.

In [ ]:
chest_test = Path('/content/chest_xray/test')
shutil.move(str(chest_test) , str(IMAGE_PATH))

'/content/Imagenes Pulmones/test'

In [ ]:
im_normal_test = Path('/content/Imagenes Pulmones/test/NORMAL')
im_neumonia_test = Path('/content/Imagenes Pulmones/test/PNEUMONIA')

In [ ]:
chest_val_normal = Path('/content/chest_xray/val/NORMAL')
chest_train_normal =  Path('/content/chest_xray/train/NORMAL')
chest_val_pneumonia = Path('/content/chest_xray/val/PNEUMONIA')
chest_train_pneumonia =  Path('/content/chest_xray/train/PNEUMONIA')

In [ ]:
im_val_normal = get_image_files(chest_val_normal)
im_train_normal = get_image_files(chest_train_normal)
im_normal = im_val_normal + im_train_normal

In [ ]:
im_val_pneumonia = get_image_files(chest_val_pneumonia)
im_train_pneumonia = get_image_files(chest_train_pneumonia)
im_pneumonia = im_val_pneumonia + im_train_pneumonia

In [ ]:
for im in im_normal:
  shutil.move(str(im),str(IM_PATH_NORMAL))
for im in im_pneumonia:
  shutil.move(str(im),str(IM_PATH_NEUMONIA))

###Se crea la carpeta donde se exportara el modelo

In [ ]:
modelo = Path('/content/drive/MyDrive/tfg/modelo')
modelo.mkdir(parents=True, exist_ok=True)

###Cantidad de imágenes de imágenes 

In [ ]:
print(f"Fotos de pulmones sanos: {len(im_normal)}")
print(f"Fotos de pulmones con neumonia: {len(im_pneumonia)}")
print(f"Fotos de pulmones sanos en test: {len(get_image_files(im_normal_test))}")
print(f"Fotos de pulmones con neumonia en test: {len(get_image_files(im_neumonia_test))}")


Fotos de pulmones sanos: 1349
Fotos de pulmones con neumonia: 3883
Fotos de pulmones sanos en test: 234
Fotos de pulmones con neumonia en test: 390


###Cambiar el nombre de las imagenes:

In [ ]:
l=get_image_files(IMAGE_PATH)

In [ ]:
identificador = 0
for im in sorted(list(l)):
  nuevo_nombre = f"{im.parents[0]}/{str(identificador).zfill(5)}.jpg"
  identificador = identificador + 1
  os.rename(str(im), nuevo_nombre)

#MODEL OPTIONS

In [ ]:
BS = 20
EPOCHS = 20
WD = 1e-1
ITEM_IMAGE_SIZE = 224 

#MODEL

Se empieza definiendo el modelo y las especificaciones que tendrá el dataset.

In [ ]:
chest = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items= get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y= parent_label,
    item_tfms=Resize(ITEM_IMAGE_SIZE)
    )

In [ ]:
dls = chest.dataloaders(IMAGES_FOR_MODEL, bs=BS, wd=WD)
#len(dls.dataset)

In [ ]:
dls.show_batch()

##Se utiliza las redes neuronales convolucionales como extractoras de patrones.

In [ ]:
learn = cnn_learner(dls, alexnet,
                    metrics=accuracy
                    )
                    #<- error_rate and accuracy are for single label classification problems
                    

##Fine-tuning

In [ ]:
learn.fine_tune(EPOCHS)

#INTERPRETACIÓN DE RESULTADOS

In [ ]:
interp= ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(8, figsize=(15,11))

# EXPORT

In [ ]:
learn.export(modelo/'resnet50_modelo.pkl')